In [1]:
import sys
import polars as pl
from pathlib import Path

sys.path.insert(0, str(Path("../../").resolve()))
# Path relativo alla root del progetto (2 livelli sopra da strategies/example/)
data_dir = Path("../../database") 

from engine.datafeed import DataFeed, MarketData
from engine.engine import *
from engine.factor import *
from engine.plotting import *
from engine.reports import *


In [2]:
# PRICES_PATH = str(project_root / 'data')
universe = ['SX5E']
start_date='2026-02-01'
end_date='2026-12-31'

# cfg = Config(universe=universe, start_date=start_date, end_date=end_date, frequency="eod")
feed = DataFeed(prices_base=data_dir)

In [3]:
factors = [ATR(5),
            DonchianChannels(20)]
            
researcher = Researcher(factors,
                        feed,
                        start_date, end_date,
                        frequency="eod",
                        tickers=universe)

In [4]:
# fig = researcher.plot(ticker="SX5E")  # Plotta automaticamente tutti i fattori
# fig.show()

In [5]:
class TestStrategy(Strategy):

    def __init__(self) -> None:
        super().__init__()
        self.ticker = 'SX5E'
        self.factors = [
            ATR(5),
            DonchianChannels(20)
        ]
        self.position_open = False

    def on_bar(self):
        if self.period <= 1:
            return
        close = self.market_data.get(self.ticker, 0)['close'].item()
        prev_close = self.market_data.get(self.ticker, 1)['close'].item()
        dc_upper = self.market_data.get(self.ticker, 1)['dc_20_upper'].item()
        prev_dc_upper = self.market_data.get(self.ticker, 2)['dc_20_upper'].item()
        dc_lower = self.market_data.get(self.ticker, 1)['dc_20_lower'].item()
        prev_dc_lower = self.market_data.get(self.ticker, 2)['dc_20_lower'].item()
        size = self.portfolio.cash / close

        # Open position
        if close > dc_lower and prev_close < prev_dc_lower and not self.position_open:
            self.buy(self.ticker, size)
            self.position_open = True

        # Close position
        if self.position_open:
            if close < dc_upper and prev_close > prev_dc_upper:
                self.close_position(self.ticker)
                self.position_open = False
            elif close < dc_lower:
                self.close_position(self.ticker)
                self.position_open = False


In [6]:
starting_balance = 100000.

exec_model = ExecutionModel(mode="on_close", slippage_bps=0.0, commission_bps=0.0)
risk = RiskManager(max_leverage=4)
reporter = ReportWriter(out_dir="backtest_reports/notebook_demo")

In [7]:
strat = TestStrategy()
strat.backtest_refactor(starting_balance, start_date, end_date, "1m", universe, feed, exec_model, risk, reporter, intraday_log=True)

15:41:19 [Strategy.TestStrategy] INFO: ============================================================
15:41:19 [Strategy.TestStrategy] INFO: BACKTEST STARTED
15:41:19 [Strategy.TestStrategy] INFO: ============================================================
15:41:19 [Strategy.TestStrategy] INFO: Period: 2026-02-01 → 2026-12-31
15:41:19 [Strategy.TestStrategy] INFO: Universe: SX5E
15:41:19 [Strategy.TestStrategy] INFO: Starting Balance: $100,000.00
15:41:19 [Strategy.TestStrategy] INFO: Execution Mode: on_close
15:41:19 [Strategy.TestStrategy] INFO: Slippage: 0.0 bps | Commission: 0.0 bps
15:41:19 [Strategy.TestStrategy] INFO: ------------------------------------------------------------
15:41:19 [Strategy.TestStrategy] INFO: ORDER SUBMITTED → SX5E | LONG 17 shares | Type: MKT @ 2026-02-02 10:03:00
15:41:19 [Strategy.TestStrategy] INFO: ORDER FILLED → SX5E | LONG 17 @ $5920.32 | Notional: $100,000.00 @ 2026-02-02 10:03:00
15:41:19 [Strategy.TestStrategy] INFO: POSITION CLOSED → SX5E @ 2026

In [9]:
self = StrategyAnalytics(strat)

In [10]:
self.summary()

{'Expected Return': '0.002691',
 'Std Deviation': '0.004860',
 'Sharpe Ratio': '0.553823',
 'N. Trade': 132,
 'Win Rate': '0.267468',
 'Average P&L': '13.790770',
 'Average Win': '164.214763',
 'Average Loss': '-41.133163',
 'Max Profit': '595.007812',
 'Max Loss': '-135.143734',
 'Profit Factor': '1.457688',
 'Max Drawdown': '-0.003859'}

In [11]:
self.daily_equity

,ref_date,time,daily_equity
0,2026-02-02,09:19:00,100000.000000
1,2026-02-02,17:49:00,100917.994883
2,2026-02-03,17:49:00,101181.348722
3,2026-02-04,17:49:00,100896.255761
4,2026-02-05,17:49:00,100790.890549
5,2026-02-06,17:49:00,101348.222721


In [ ]:
self.plot_balance("SX5E")

In [ ]:
analytics = StrategyAnalytics(self)

AttributeError: 'StrategyAnalytics' object has no attribute 'positions_summary'

In [ ]:
analytics.summary()

KeyError: 'trade_id'

In [ ]:
closed_trades = analytics.trades_df[analytics.trades_df['is_alive'] == False]

In [ ]:
closed_trades

,ref_date,ref_time,entry_time,symbol,type,side,quantity,entry_price,is_alive,current_value,open_pnl,closed_pnl,commissions,global_pnl
0,2026-02-02,10:04:00,2026-02-02 10:03:00,SX5E,equity,1,0.0,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
0,2026-02-02,10:05:00,2026-02-02 10:03:00,SX5E,equity,1,0.0,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
0,2026-02-02,10:06:00,2026-02-02 10:03:00,SX5E,equity,1,0.0,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
0,2026-02-02,10:07:00,2026-02-02 10:03:00,SX5E,equity,1,0.0,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
0,2026-02-02,10:08:00,2026-02-02 10:03:00,SX5E,equity,1,0.0,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2026-02-06,17:49:00,2026-02-06 14:57:00,SX5E,equity,1,0.0,5958.95,False,0.0,0.0,-27.454572,0.0,-27.454572
128,2026-02-06,17:49:00,2026-02-06 15:01:00,SX5E,equity,1,0.0,5957.58,False,0.0,0.0,79.648810,0.0,79.648810
129,2026-02-06,17:49:00,2026-02-06 16:07:00,SX5E,equity,1,0.0,5976.36,False,0.0,0.0,109.385896,0.0,109.385896
130,2026-02-06,17:49:00,2026-02-06 16:51:00,SX5E,equity,1,0.0,5985.07,False,0.0,0.0,-19.435348,0.0,-19.435348


In [ ]:
analytics.trades_df

,ref_date,ref_time,entry_time,symbol,type,side,quantity,entry_price,is_alive,current_value,open_pnl,closed_pnl,commissions,global_pnl
0,2026-02-02,10:03:00,2026-02-02 10:03:00,SX5E,equity,1,16.890979,5920.32,True,100000.0,0.0,-0.000000,0.0,0.000000
0,2026-02-02,10:04:00,2026-02-02 10:03:00,SX5E,equity,1,0.000000,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
0,2026-02-02,10:05:00,2026-02-02 10:03:00,SX5E,equity,1,0.000000,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
0,2026-02-02,10:06:00,2026-02-02 10:03:00,SX5E,equity,1,0.000000,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
0,2026-02-02,10:07:00,2026-02-02 10:03:00,SX5E,equity,1,0.000000,5920.32,False,0.0,0.0,-32.092860,0.0,-32.092860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2026-02-06,17:49:00,2026-02-06 14:57:00,SX5E,equity,1,0.000000,5958.95,False,0.0,0.0,-27.454572,0.0,-27.454572
128,2026-02-06,17:49:00,2026-02-06 15:01:00,SX5E,equity,1,0.000000,5957.58,False,0.0,0.0,79.648810,0.0,79.648810
129,2026-02-06,17:49:00,2026-02-06 16:07:00,SX5E,equity,1,0.000000,5976.36,False,0.0,0.0,109.385896,0.0,109.385896
130,2026-02-06,17:49:00,2026-02-06 16:51:00,SX5E,equity,1,0.000000,5985.07,False,0.0,0.0,-19.435348,0.0,-19.435348


In [ ]:
# fig = analytics.plot_price_with_signals(
#     'SX5E',
#     plot_factors=['dc_20_upper', 'dc_20_lower'],
#     chart_type='candlestick'  # Use candlestick chart instead of line chart
# )
# fig.show()


In [ ]:
analytics.plot_balance("SX5E")

# DataFeed DataFrame

In [ ]:
# Carica i dati di mercato usando DataFeed
df = feed.get_market_data(start_date, end_date, 'eod', universe)
print(f"Shape: {df.shape}")
print(f"\nPrime righe:")
print(df.head())


Shape: (537, 10)

Prime righe:
shape: (5, 10)
┌────────────┬──────────┬────────┬─────────┬───┬─────────┬──────────────┬────────────────┬──────┐
│ date       ┆ time     ┆ ticker ┆ open    ┆ … ┆ close   ┆ volume       ┆ insertion_time ┆ type │
│ ---        ┆ ---      ┆ ---    ┆ ---     ┆   ┆ ---     ┆ ---          ┆ ---            ┆ ---  │
│ date       ┆ time     ┆ str    ┆ f64     ┆   ┆ f64     ┆ f64          ┆ datetime[μs]   ┆ str  │
╞════════════╪══════════╪════════╪═════════╪═══╪═════════╪══════════════╪════════════════╪══════╡
│ 2024-01-02 ┆ 01:00:00 ┆ SX5E   ┆ 4528.45 ┆ … ┆ 4512.81 ┆ 2.80750176e8 ┆ 2026-02-03     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 16:40:00       ┆      │
│ 2024-01-03 ┆ 01:00:00 ┆ SX5E   ┆ 4514.2  ┆ … ┆ 4448.13 ┆ 2.66292508e8 ┆ 2026-02-03     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 16:40:00       ┆      │
│ 2024-01-04 ┆ 01:00:00 ┆ SX5E   ┆ 4451.72 ┆ … ┆ 4474.01 ┆ 2.8974942e8  

In [ ]:
# feed.get_data_excel(start_date, end_date, '1m', universe)

In [ ]:
# Crea MarketData dal DataFrame storico
market_data = MarketData.from_dataframe(df)
print(f"MarketData creato con {len(market_data.symbols())} simboli")
print(f"Current date: {market_data.current_date()}")
print(f"Current time: {market_data.current_time()}")


MarketData creato con 1 simboli
Current date: 2026-02-04
Current time: 01:00:00


In [ ]:
# Esempi di accesso ai prezzi
print("Prezzo corrente:")
print(f"  SX5E close: ${market_data.price('SX5E', 'close'):.2f}")

print("\nPrezzo T-1 (1 bar fa):")
print(f"  SX5E close: ${market_data.price('SX5E', 'close', 1):.2f}")

print("\nUltimi 10 prezzi:")
last_10 = market_data.price('SX5E', 'close', slice(0, 10))
print(f"  Shape: {len(last_10)} valori")
print(f"  Ultimi 3: {last_10[-3:].to_list()}")


Prezzo corrente:
  SX5E close: $5970.47

Prezzo T-1 (1 bar fa):
  SX5E close: $5995.35

Ultimi 10 prezzi:
  Shape: 10 valori
  Ultimi 3: [6007.51, 5995.35, 5970.47]
